In [0]:
!wget http://www.gutenberg.org/files/11/11-0.txt -O ./data/alice_in_wonderland.txt

In [4]:
from __future__ import print_function

import numpy as np
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
import codecs


INPUT_FILE = "./data/alice_in_wonderland.txt"

# extract the input as a stream of characters
print("Extracting text from input...")
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as f:
    lines = [line.strip().lower() for line in f
             if len(line) != 0]
    text = " ".join(lines)

# creating lookup tables
# Here chars is the number of features in our character "vocabulary"
chars = set(text)
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

# create inputs and labels from the text. We do this by stepping
# through the text ${step} character at a time, and extracting a
# sequence of size ${seqlen} and the next output char. For example,
# assuming an input text "The sky was falling", we would get the
# following sequence of input_chars and label_chars (first 5 only)
#   The sky wa -> s
#   he sky was ->
#   e sky was  -> f
#    sky was f -> a
#   sky was fa -> l
print("Creating input and label text...")
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])
    
print('input_0: ', input_chars[0])
print('label_0: ', label_chars[0])

# vectorize the input and label chars
# Each row of the input is represented by seqlen characters, each
# represented as a 1-hot encoding of size len(char). There are
# len(input_chars) such rows, so shape(X) is (len(input_chars),
# seqlen, nb_chars).
# Each row of output is a single character, also represented as a
# dense encoding of size len(char). Hence shape(y) is (len(input_chars),
# nb_chars).
print("Vectorizing input and label text...")
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

Extracting text from input...
Creating input and label text...
input_0:  ﻿project g
label_0:  u
Vectorizing input and label text...


In [5]:
# Build the model. We use a single RNN with a fully connected layer
# to compute the most likely predicted output char
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.summary()

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               24192     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7740      
_________________________________________________________________
activation_1 (Activation)    (None, 60)                0         
Total params: 31,932
Trainable params: 31,932
Non-trainable params: 0
_________________________________________________________________


In [6]:
# We train the model in batches and test output generated at each step
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: {}".format(iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # testing model
    # randomly choose a row from input_chars, then use it to
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: {}".format(test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            Xtest[0, j, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()

Iteration #: 0
Epoch 1/1
162739/162739 [==============================] - 11s 69us/step - loss: 2.3704
Generating from seed: , who had 
, who had the the the the the the the the the the the the the the the the the the the the the the the the the 
Iteration #: 1
Epoch 1/1
162739/162739 [==============================] - 10s 64us/step - loss: 2.0625
Generating from seed: atching th
atching the she said the dore all at in the said the dore all at in the said the dore all at in the said the d
Iteration #: 2
Epoch 1/1
162739/162739 [==============================] - 10s 63us/step - loss: 1.9536
Generating from seed: t you, wil
t you, will a don the hat ere the mouthe her her her her her her her her her her her her her her her her her h
Iteration #: 3
Epoch 1/1
162739/162739 [==============================] - 11s 65us/step - loss: 1.8693
Generating from seed: e on the l
e on the liotle would be the way soo she was a lictle soo she was a lictle soo she was a lictle soo she was a 
Iteration #: